### Extract only MCC

In [2]:
import pandas as pd
#dev
import importlib as imp

import functions
imp.reload(functions)

from functions import *

import librosa

import os

import numpy as np

In [3]:
BASE_PATH = "datasets/DAIC-WOZ/"
files = os.listdir(BASE_PATH+"/cuts")
df_train = pd.read_csv(BASE_PATH + "train_split_Depression_AVEC2017.csv")
df_dev = pd.read_csv(BASE_PATH + "dev_split_Depression_AVEC2017.csv")
df_dev = df_dev[['Participant_ID', 'PHQ8_Binary', 'PHQ8_Score']]
df_train = df_train[['Participant_ID', 'PHQ8_Binary', 'PHQ8_Score']]

In [6]:
def process_segment(file_path, sr=16000):
    """
    Process each valid speech segment
    """

    audio, sr = librosa.load(file_path, sr=sr)


    mel_spect = librosa.feature.melspectrogram(y=audio, sr=sr, n_mels=40)
    # Convert to log scale (this is our MCC representation)
    mccs = librosa.power_to_db(mel_spect)
    
    # Normalize each feature
    mccs = (mccs - mccs.mean(axis=1, keepdims=True)) / (mccs.std(axis=1, keepdims=True) + 1e-10)
    
    return mccs

def get_fixed_length_segments(mfcc, segment_length=200):
    """
    Ensure each MFCC has fixed length by either padding or truncating
    """
    current_length = mfcc.shape[1]
    
    if current_length > segment_length:
        # Take center part if too long
        start = (current_length - segment_length) // 2
        return mfcc[:, start:start+segment_length]
    elif current_length < segment_length:
        # Pad with zeros if too short
        padded = np.zeros((mfcc.shape[0], segment_length))
        start = (segment_length - current_length) // 2
        padded[:, start:start+current_length] = mfcc
        return padded
    else:
        return mfcc

In [ ]:
# Process all files
base_path = BASE_PATH + "cuts"
all_mccs = []
all_labels = []

for _, row in df_dev.iterrows():
    patient = row['Participant_ID']
    print(patient)
    label = row['PHQ8_Binary']  # Assuming this is your label
    
    patient_files = [f for f in os.listdir(base_path) 
                    if f.startswith(str(patient)) and f.endswith('.wav')]
    
    for f in patient_files:
        file_path = f"{base_path}/{f}"
        try:
            # Extract MFCC
            mcc = process_segment(file_path)
            
            # Ensure fixed length
            mcc = get_fixed_length_segments(mcc, segment_length=200)
            
            # Store MFCC and label
            all_mccs.append(mcc)
            all_labels.append(label)
            
            # Print shapes occasionally to verify
            if len(all_mccs) % 100 == 0:
                print(f"Processed {len(all_mccs)} segments. Current MCC shape: {mcc.shape}")
                
        except Exception as e:
            print(f"Error processing {file_path}: {str(e)}")
            continue

# Convert to numpy arrays
X = np.array(all_mccs)
y = np.array(all_labels)

# save X and y into a .npz file
np.savez_compressed('DAIC_mcc_test.npz', X=X, y=y)

print(f"Final dataset shape: {X.shape}")
print(f"Labels shape: {y.shape}")

303
304
Processed 100 segments. Current MFCC shape: (40, 200)
305
Processed 200 segments. Current MFCC shape: (40, 200)
310
312
Processed 300 segments. Current MFCC shape: (40, 200)
313
Processed 400 segments. Current MFCC shape: (40, 200)
315
316
Processed 500 segments. Current MFCC shape: (40, 200)
317
Processed 600 segments. Current MFCC shape: (40, 200)
318
319
Processed 700 segments. Current MFCC shape: (40, 200)
320
Processed 800 segments. Current MFCC shape: (40, 200)
321
322
Processed 900 segments. Current MFCC shape: (40, 200)
324
Processed 1000 segments. Current MFCC shape: (40, 200)
325
326
Processed 1100 segments. Current MFCC shape: (40, 200)
327
Processed 1200 segments. Current MFCC shape: (40, 200)
328
330
Processed 1300 segments. Current MFCC shape: (40, 200)
333
336
Processed 1400 segments. Current MFCC shape: (40, 200)
338
Processed 1500 segments. Current MFCC shape: (40, 200)
339
340
Processed 1600 segments. Current MFCC shape: (40, 200)
341
343
Processed 1700 segmen